In [1]:
from pathlib import Path

import typer
import pandas as pd
import verifiers as vf
from dotenv import load_dotenv
from openai import AsyncOpenAI


assert load_dotenv(), "Failed to load .env file"

openai_client = AsyncOpenAI()

In [2]:
import mlflow

# Tell MLflow about the server URI.
mlflow.set_tracking_uri("http://127.0.0.1:5000")
# Enable autologging with all features
mlflow.openai.autolog()
# Create a unique name for your experiment.
mlflow.set_experiment("rlvr-eval")

<Experiment: artifact_location='mlflow-artifacts:/2', creation_time=1756636926792, experiment_id='2', last_update_time=1756636926792, lifecycle_stage='active', name='rlvr-eval', tags={}>

In [3]:
from rlvr.utils.llm import get_default_model
model = get_default_model()
model

'Qwen/Qwen2.5-7B-Instruct'

In [4]:

env_id = "vf-musique-structured"
datasets_str = "bdsaglam/musique-mini,answerable,validation"
noise_rate = 1.0
retriever = "hybrid"
temperature = 0.5
max_new_tokens = 1024

print("🔮 Starting MuSiQue evaluation")
print("=" * 50)
print(f"📝 Model: {model}")
print(f"📊 Dataset: {datasets_str} (noise rate: {noise_rate})")
print(f"🔍 Retriever: {retriever}")
print(f"🌡️ Temperature: {temperature}")
print(f"🎯 Max tokens: {max_new_tokens}")
print("=" * 50)

# Load MuSiQue environment
print("🌍 Loading MuSiQue environment...")

vf_env = vf.load_environment(
    env_id="vf-musique",
    datasets_str=datasets_str,
    noise_rate=noise_rate,
    retriever=retriever,
)
print(f"✅ Environment loaded with {len(vf_env.dataset)} examples")

# Use OpenAI-compatible API client (e.g., for vLLM)
print("🤖 Using OpenAI-compatible API client...")

# Run evaluation using the environment
print("🔄 Running evaluation...")
results = vf_env.evaluate(
    openai_client,
    model,
    rollouts_per_example=1,
    sampling_args={"temperature": temperature, "max_tokens": max_new_tokens},
)


🔮 Starting MuSiQue evaluation
📝 Model: Qwen/Qwen2.5-7B-Instruct
📊 Dataset: bdsaglam/musique-mini,answerable,validation (noise rate: 1.0)
🔍 Retriever: hybrid
🌡️ Temperature: 0.5
🎯 Max tokens: 1024
🌍 Loading MuSiQue environment...


Map: 100%|##########| 300/300 [00:00<?, ? examples/s]

Filter:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

2025-09-14 18:14:15 - verifiers.envs.MuSiQueEnv - INFO - eval_dataset is not set, falling back to train dataset


✅ Environment loaded with 100 examples
🤖 Using OpenAI-compatible API client...
🔄 Running evaluation...


Evaluating 100 rollouts: 100%|██████████| 100/100 [00:00<00:00, 843.93it/s]


[Trace(trace_id=tr-a76e5138155bfe95844a0edd84b6e010), Trace(trace_id=tr-55bd2d2dddbadade7cf7ef8a1e9283ee), Trace(trace_id=tr-c0a50514a5f55c4688e75e837a2a5900), Trace(trace_id=tr-a77dc6050f199d6bc837674278c561d6), Trace(trace_id=tr-5928c30d89a6d9b9f39090a5fef3a1be), Trace(trace_id=tr-1af57b33a906ed25ff261f25e12eda6d), Trace(trace_id=tr-59db161fca220f4efe6a42bc43c38a33), Trace(trace_id=tr-6fe07ff8eb36d75d97c4db0262051557), Trace(trace_id=tr-4b425fae375cc15c6f6dc21777cd1b5e), Trace(trace_id=tr-dbefc1befb636c662c95134cd4ebb32a)]

In [5]:
from datetime import datetime
import json

exp_id = datetime.now().strftime("%Y%m%d_%H%M%S")
exp_dir = Path(f"../outputs/musique-eval/{exp_id}")
exp_dir.mkdir(parents=True, exist_ok=True)

params = {
    "env_id": env_id,
    "datasets_str": datasets_str,
    "noise_rate": noise_rate,
    "retriever": retriever,
    "model": model,
    "temperature": temperature,
    "max_new_tokens": max_new_tokens,
}

(exp_dir / "params.json").write_text(json.dumps(params, indent=2))

237

In [6]:
ds = vf_env.make_dataset(results)
ds

Dataset({
    features: ['prompt', 'completion', 'answer', 'task', 'reward', 'info', 'exact_match_reward', 'f1_reward', 'retrieval_recall_reward', 'retrieval_precision_reward', 'citation_reward', 'format_reward', 'combined_reward'],
    num_rows: 100
})

In [7]:
ds.to_json(exp_dir / "musique-eval-results.jsonl", orient="records", lines=True)

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

2801680

In [8]:
df = ds.to_pandas()
df

,prompt,completion,answer,task,reward,info,exact_match_reward,f1_reward,retrieval_recall_reward,retrieval_precision_reward,citation_reward,format_reward,combined_reward
0,[{'content': 'Answer the question based on the...,"[{'content': '<think> To answer this question,...",LeMat revolver,default,0.000000,"{'answers': ['LeMat Revolver', 'LeMat revolver...",0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
1,[{'content': 'Answer the question based on the...,"[{'content': '<think> To answer this question,...",Vitor Constâncio,default,0.516667,"{'answers': ['Vitor Constâncio', 'vitor constâ...",0.0,0.0,1.0,1.000000,1.0,0.666667,0.516667
2,[{'content': 'Answer the question based on the...,"[{'content': '', 'role': 'assistant', 'tool_ca...",45,default,0.175000,"{'answers': ['45'], 'docs': [{'body': 'Weber C...",0.0,0.0,0.5,0.500000,0.0,0.000000,0.175000
3,[{'content': 'Answer the question based on the...,"[{'content': '<think> To answer this question,...",three different relationships he had in the past,default,0.375000,{'answers': ['three different relationships he...,0.0,0.0,1.0,1.000000,0.0,1.000000,0.375000
4,[{'content': 'Answer the question based on the...,"[{'content': '<think> To answer this question,...",Michael Bublé,default,0.166667,"{'answers': ['michael bublé', 'Michael Bublé']...",0.0,0.0,0.5,0.333333,0.0,0.333333,0.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,[{'content': 'Answer the question based on the...,"[{'content': '<think> To answer this question,...",two alliances,default,0.000000,"{'answers': ['two alliances'], 'docs': [{'body...",0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
96,[{'content': 'Answer the question based on the...,"[{'content': '<think> To answer this question,...",John Kukuzelis,default,0.000000,"{'answers': ['John Kukuzelis', 'john kukuzelis...",0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
97,[{'content': 'Answer the question based on the...,"[{'content': '', 'role': 'assistant', 'tool_ca...",Vito Corleone,default,0.175000,"{'answers': ['Vito Corleone', 'vito corleone',...",0.0,0.0,0.5,0.500000,0.0,0.000000,0.175000
98,[{'content': 'Answer the question based on the...,"[{'content': '<think> To answer this question,...",The Île de la Cité,default,0.875000,"{'answers': ['the île de la cité', 'The Île de...",1.0,1.0,1.0,0.500000,0.5,1.000000,0.875000


In [9]:
df.describe()

,reward,exact_match_reward,f1_reward,retrieval_recall_reward,retrieval_precision_reward,citation_reward,format_reward,combined_reward
count,100.000000,100.00000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,0.377965,0.15000,0.209250,0.690000,0.563833,0.335000,0.570000,0.377965
std,0.252313,0.35887,0.374822,0.299663,0.248057,0.389606,0.367889,0.252313
min,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.183333,0.00000,0.000000,0.500000,0.500000,0.000000,0.333333,0.183333
50%,0.300000,0.00000,0.000000,0.500000,0.500000,0.000000,0.666667,0.300000
75%,0.516667,0.00000,0.254167,1.000000,0.666667,0.500000,1.000000,0.516667
max,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
df.describe().to_csv(exp_dir / "scores.csv")